In [78]:
from typing import Any, Dict, List

import nest_asyncio
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import (HumanMessagePromptTemplate, MessagesPlaceholder)
from langchain.schema.output_parser import StrOutputParser

from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.chat import ChatPromptTemplate


from langchain_core.runnables.base import RunnableSerializable

from langchain_openai import ChatOpenAI

from langchain_groq import ChatGroq
from rich.pretty import pprint

nest_asyncio.apply()


def pretty_print(title: str = None, content: Any = None) -> None:
    if title is None:
        print(content)
        return
    print(title)
    pprint(content)

In [79]:
import os
os.environ["LANGCHAIN_PROJECT"] = "langchain_grah" # Monitored by LangSmith

In [80]:
llm: BaseChatModel = ChatOpenAI(model="gpt-4-turbo-preview", temperature=0)  

#### Warm-up: Single-user conversation

We use `ConversationBufferMemory` and `ConversationChain` to build classical LangChain chatbot.

In [ ]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(HumanMessage(content="Hello", id="user1"))
memory.buffer.append(HumanMessage(content="oha?", id="user1"))
memory.buffer.append(HumanMessage(content="lol" , id="user1"))
memory.buffer.append(HumanMessage(content="happy" , id="user1"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)

model = llm
conversation = ConversationChain(llm=model, memory=memory)
conv_chain_out = conversation.invoke(input = """How many users are involved in this conversation?
                     Notice: Give me a simple result with the only number of users, ie. 1 or 2 or 3....""")
pretty_print("Conversation Chain Output", conv_chain_out)

conv_chain_out = conversation.invoke(
    input="""Give the list of all the messages from "user1" 
and put them in a "[]" without any instruction text, newlines or additional information.
""",
)
pretty_print("user1 conv_chain_out", conv_chain_out)

#### Multi-user conversation

We use `ConversationBufferMemory` and `ConversationChain` to build classical LangChain chatbot.

We logically use the id of `HumanMessage` to identify different users.

The data in `ConversationBufferMemory` expands with each invocation so that the memory grows automatically.

In [ ]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user_1"))
memory.buffer.append(HumanMessage(content="hi", id="user_2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user_3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user_4"))
memory.buffer.append(HumanMessage(content="glad to see you", id="user_5"))
memory.buffer.append(HumanMessage(content="good luck dudes", id="user_5"))
memory.buffer.append(HumanMessage(content="I'm a great user", id="user_5"))
memory.buffer.append(HumanMessage(content="great to see you", id="user_6"))
# memory.buffer.append(HumanMessage(content="Merci", id="user_7"))
# memory.buffer.append(HumanMessage(content="Danke sehr", id="user_8"))
memory.buffer.append(HumanMessage(content="Merci", id="user_XL"))
memory.buffer.append(HumanMessage(content="Danke sehr", id="user_XXL"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)

model = llm
conversation = ConversationChain(llm=model, memory=memory)
conv_chain_out = conversation.invoke(
    input="""How many users are involved in this conversation?
Also provide the list of user ids. The user ids can be any format unique to each user.
Use 'id' as unique identifier for each user.

Notice: 

Give me a simple result with the only number of users without any instruction text or additional information,ie. 1,2 or 3....
Output format: 
user_count=x, x is number of users

The user ids will be saved inside "[]".
Output format: 
user_ids=[user_1,.......]"""
)
pretty_print("conv_chain_out", conv_chain_out)


conv_chain_out = conversation.invoke(
    input="""Give the list of all the messages from "user_5" and put them in a "[]" without any instruction text, newlines or additional information.
""",
)
pretty_print("user_5 conv_chain_out", conv_chain_out)


# def build_chain_without_parsing(
#     model: BaseChatModel,
# ) -> RunnableSerializable[Dict, str]:
#     prompt = ChatPromptTemplate.from_messages(
#         [
#             SystemMessage(
#                 content=("You are an AI assistant." "You can handle the query of user.")
#             ),
#             MessagesPlaceholder(variable_name="history"),
#             HumanMessagePromptTemplate.from_template("{query}"),
#         ]
#     )
#     return prompt | model


# conv_chain_history = conv_chain_out["history"]
# pretty_print("conv_chain_history", conv_chain_history)


# res = (build_chain_without_parsing(model) | StrOutputParser()).invoke(
#     {
#         "history": conv_chain_history,
#         "query": """Give the list of all the messages from "user_5" and put them in a "[]" without any instruction text or additional information.
# """,
#     }
# )
# pretty_print("Result", res)

### Multi-user conversation with LCEL 

#### Failed approach: Directly use the history of `ConversationBufferMemory`: `cxt_history`

**This approach won't work**.

Assign the `ConversationBufferMemory` history to `MessagesPlaceholder(variable_name="history")` in the chain. 

However, this will not work as the model input cannot access any user-ids. 

Verify with LangSmith.

In [ ]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
# memory.buffer.append(HumanMessage(content="Merci", id="user_7"))
# memory.buffer.append(HumanMessage(content="Danke sehr", id="user_8"))
memory.buffer.append(HumanMessage(content="Merci", id="user_XL"))
memory.buffer.append(HumanMessage(content="Danke sehr", id="user_XXL"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)


cxt_history = memory.load_memory_variables({})["history"]
pretty_print("cxt_history", cxt_history)


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = llm

human_query = HumanMessage(
    content="""Calculate the unique count of 'id'.""",
    id="user-X",
)

res = build_chain_without_parsing(model).invoke(
    {
        "history": cxt_history,
        "query": human_query.content,
    }
)
pretty_print("Result", res)

# Update memory
memory.buffer.append(human_query)
memory.buffer.append(res)
cxt_history = memory.load_memory_variables({})["history"]
pretty_print("cxt_history", cxt_history)


res = (build_chain_without_parsing(model) | StrOutputParser()).invoke(
    {
        "history": cxt_history,
        "query": """What did "user-5" speak? When he spoke more than once,
give me the list of messages of "user-5" and put all messages of "user-5" in '[]' without any instruction text, newlines or additional information.
""",
    }
)
pretty_print("Result", res)

#### Convert all history into a single string: `cxt_string`

We can compress all history messages, including system and AI messages, into a single string and provide it to the model as history.

All historical information messages are combined into a single string, including the user's role, user ID, and message content, a limitation of the LangSmith overview, ie:

```python
ChatPromptValue(
│   messages=[
│   │   SystemMessage(content='You are an AI assistant.You can handle the query of user.'),
│   │   HumanMessage(
│   │   │   content='The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. \nIf the AI does not know the answer to a question, it truthfully says it does not know.\n\nNotice: The \'uid\' is user id, \'role\' is user role for human or ai, \'content\' is the message content.\n{\n{\n"uid":"",\n"role":"ai",\n"content": "This is a Gaming Place"\n},\n{\n"uid":"user-1",\n"role":"human",\n"content": "Hello dudes"\n},\n{\n"uid":"user-2",\n"role":"human",\n"content": "hi"\n},\n{\n"uid":"user-3",\n"role":"human",\n"content": "yo yo"\n},\n{\n"uid":"user-4",\n"role":"human",\n"content": "nice to see you"\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "hoho dude"\n},\n{\n"uid":"user-L",\n"role":"human",\n"content": "o lalala"\n},\n{\n"uid":"user-XXXXL",\n"role":"human",\n"content": "guten tag"\n},\n{\n"uid":"user-1",\n"role":"human",\n"content": "Let\'s get started, ok?"\n},\n{\n"uid":"user-2",\n"role":"human",\n"content": "YES"\n},\n{\n"uid":"user-3",\n"role":"human",\n"content": "YEAH...."\n},\n{\n"uid":"user-4",\n"role":"human",\n"content": "Cool.."\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "yup."\n},\n{\n"uid":"user-L",\n"role":"human",\n"content": "Great....."\n},\n{\n"uid":"user-XXXXL",\n"role":"human",\n"content": "alles klar"\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "I am the winner"\n}\n}'
│   │   ),
│   │   HumanMessage(
│   │   │   content='content=\'How many users are involved in this conversation exclude the AI or System messages?\\nAlso provide the list of user ids. The user ids can be any format unique to each user.\\n\\nNotice: \\n\\nGive me a simple result with the only number of users without any instruction text or additional information,\\nkeep the result as simple as possible,ie. 1,2 or 3....\\nOutput format: \\nuser_count=x, x is number of users\\n\\nThe user ids will be saved inside "[]".\\nOutput format: \\nuser_ids=[user_1,.......]\' id=\'user-X\''
│   │   )
│   ]
)
```

Check `convert_memory_to_str()` for more details.

Different from  `ConversationBufferMemory` and `ConversationChain` , the memory will not grow with each invocation,

we need to code the logic to update the memory.


In [130]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
memory.buffer.append(HumanMessage(content="alles klar", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="I am the winner", id="user-5"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)


def convert_memory_to_str(memory: ConversationBufferMemory) -> str:
    """Convert the memory to str"""
    res = """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Notice: The 'uid' is user id, 'role' is user role for human or ai, 'content' is the message content.
{
"""
    history = memory.load_memory_variables({})["history"]
    for hist_item in history:
        role = "human" if isinstance(hist_item, HumanMessage) else "ai"
        res += f"""{{
"uid":"{hist_item.id if role =='human' else ''}",
"role":"{role}",
"content": "{hist_item.content}"
}},
"""
    # remove the last comma
    res = res[:-2]
    res += """
}"""
    return res


cxt_str = [convert_memory_to_str(memory)]
pretty_print("cxt_str", cxt_str)


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = llm

human_query = HumanMessage(
    content="""How many users are involved in this conversation exclude the AI or System messages?
Also provide the list of user ids. The user ids can be any format unique to each user.

Notice: 

Give me a simple result with the only number of users without any instruction text or additional information,
keep the result as simple as possible,ie. 1,2 or 3....
Output format: 
user_count=x, x is number of users

The user ids will be saved inside "[]".
Output format: 
user_ids=[user_1,.......]""",
    id="user-X",
)
res = build_chain_without_parsing(model).invoke(
    {
        "history": cxt_str,
        "query": human_query.content,
    }
)
pretty_print("Result", res)

# Update memory
memory.buffer.append(human_query)
memory.buffer.append(res)
cxt_str = [convert_memory_to_str(memory)]
pretty_print("cxt_str", cxt_str)

res = (build_chain_without_parsing(model) | StrOutputParser()).invoke(
    {
        "history": cxt_str,
        "query": """Give the list of all the messages from "user-5" and put them in a "[]" without any instruction text, newlines or additional information.
Notice: Only need content and plan-text.
""",
    }
)
pretty_print("user-5 result", res)

Memory Variables init


{'history': []}

Memory Variables in str list (buffer_as_str) init


''

Memory Variables seeded


...

AIMessage(content='user_count=7\nuser_ids=[user-1, user-2, user-3, user-4, user-5, user-L, user-XXXXL]')

cxt_str


[
│   'The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. \nIf the AI does not know the answer to a question, it truthfully says it does not know.\n\nNotice: The \'uid\' is user id, \'role\' is user role for human or ai, \'content\' is the message content.\n{\n{\n"uid":"",\n"role":"ai",\n"content": "This is a Gaming Place"\n},\n{\n"uid":"user-1",\n"role":"human",\n"content": "Hello dudes"\n},\n{\n"uid":"user-2",\n"role":"human",\n"content": "hi"\n},\n{\n"uid":"user-3",\n"role":"human",\n"content": "yo yo"\n},\n{\n"uid":"user-4",\n"role":"human",\n"content": "nice to see you"\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "hoho dude"\n},\n{\n"uid":"user-L",\n"role":"human",\n"content": "o lalala"\n},\n{\n"uid":"user-XXXXL",\n"role":"human",\n"content": "guten tag"\n},\n{\n"uid":"user-1",\n"role":"human",\n"content": "Let\'s get started, ok?"\n},\n{\n"uid":"user-2",\n"role":"human",\n"content": "YES"\n},\n{\n"uid":"user-3",\n"role":"human",\n"content": "YEAH...."\n},\n{\n"uid":"user-4",\n"role":"human",\n"content": "Cool.."\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "yup."\n},\n{\n"uid":"user-L",\n"role":"human",\n"content": "Great....."\n},\n{\n"uid":"user-XXXXL",\n"role":"human",\n"content": "alles klar"\n},\n{\n"uid":"user-5",\n"role":"human",\n"content": "I am the winner"\n},\n{\n"uid":"user-X",\n"role":"human",\n"content": "How many users are involved in this conversation exclude the AI or System messages?\nAlso provide the list of user ids. The user ids can be any format unique to each user.\n\nNotice: \n\nGive me a simple result with the only number of users without any instruction text or additional information,\nkeep the result as simple as possible,ie. 1,2 or 3....\nOutput format: \nuser_count=x, x is number of users\n\nThe user ids will be saved inside "[]".\nOutput format: \nuser_ids=[user_1,.......]"\n},\n{\n"uid":"",\n"role":"ai",\n"content": "user_count=7\nuser_ids=[user-1, user-2, user-3, user-4, user-5, user-L, user-XXXXL]"\n}\n}'
]

user-5 result


'["hoho dude", "yup.", "I am the winner"]'

#### Convert all history into a list of dictionaries: `cxt_dict`

We can compress all historical messages, including system and AI messages, into a list of dictionaries and feed it to the model as history.

The content of each block (dictionary) are the role of `user`, `user-id`, and the `message content`.

Refer to `convert_memory_to_dict()` for additional information.

Also the memory will not grow with each invocation, we need to code the logic to update the memory.

**Great advantages:**

- Clean overview in the LangSmith interface due to a well-filled prompt, see below.
    - For the input of LLM. 

- Each block of history can be converted to  AIMessage and HumanMessage for the filled prompt:

```python
ChatPromptValue(
│   messages=[
│   │   SystemMessage(content='You are an AI assistant.You can handle the query of user.'),
│   │   HumanMessage(
│   │   │   content="The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. \nIf the AI does not know the answer to a question, it truthfully says it does not know.\n\nNotice: The 'uid' is user id, 'role' is user role for human or ai, 'content' is the message content.\n\n"
│   │   ),
│   │   AIMessage(content='{\nuid:""\nrole:"ai"\ncontent:"This is a Gaming Place"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-1"\nrole:"human"\ncontent:"Hello dudes"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-2"\nrole:"human"\ncontent:"hi"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-3"\nrole:"human"\ncontent:"yo yo"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-4"\nrole:"human"\ncontent:"nice to see you"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"human"\ncontent:"hoho dude"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-L"\nrole:"human"\ncontent:"o lalala"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-XXXXL"\nrole:"human"\ncontent:"guten tag"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-1"\nrole:"human"\ncontent:"Let\'s get started, ok?"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-2"\nrole:"human"\ncontent:"YES"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-3"\nrole:"human"\ncontent:"YEAH...."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-4"\nrole:"human"\ncontent:"Cool.."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"human"\ncontent:"yup."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-L"\nrole:"human"\ncontent:"Great....."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-XXXXL"\nrole:"human"\ncontent:"alles klar"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"human"\ncontent:"I\'m good and the best."\n}\n'),
│   │   HumanMessage(
│   │   │   content='content=\'How many users are involved in this conversation exclude the AI or System messages?\\nAlso provide the list of user ids. The user ids can be any format unique to each user.\\n\\nNotice: \\n\\nGive me a simple result with the only number of users without any instruction text or additional information,\\nkeep the result as simple as possible,ie. 1,2 or 3....\\nOutput format: \\nuser_count=x, x is number of users\\n\\nThe user ids will be saved inside "[]".\\nOutput format: \\nuser_ids=[user_1,.......]\' id=\'user-X\''
│   │   )
│   ]
)
```


In [154]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
memory.buffer.append(HumanMessage(content="alles klar", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="I'm good and the best.", id="user-5"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)


def convert_memory_to_dict(memory: ConversationBufferMemory) -> List[Dict[str, str]]:
    """Convert the memory to the dict, role is id, content is the message content."""
    res = [
        """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Notice: The 'uid' is user id, 'role' is user role for human or ai, 'content' is the message content.

"""
    ]
    history = memory.load_memory_variables({})["history"]
    content_fmt = """{{
uid:"{uid}"
role:"{role}"
content:"{content}"
}}
"""
    for hist_item in history:
        role = "human" if isinstance(hist_item, HumanMessage) else "ai"
        res.append(
            {
                "role": role,
                "content": content_fmt.format(
                    content=hist_item.content,
                    uid=hist_item.id if role == "human" else "",
                    role=role,
                ),
            }
        )
    return res


cxt_dict = convert_memory_to_dict(memory)
pretty_print("cxt_dict", cxt_dict)


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = llm

human_query = HumanMessage(
    """How many users are involved in this conversation, the valid has value in 'uid'? 
Only provide the list of user `uid`s in `[... ]` as result without any instruction text or additional information.""",
    id="user-X",
)
res = build_chain_without_parsing(model).invoke(
    {
        "history": cxt_dict,
        "query": human_query.content,
    }
)
pretty_print("Result", res)

# Update memory
memory.buffer.append(human_query)
memory.buffer.append(res)
cxt_dict = convert_memory_to_dict(memory)
pretty_print("cxt_dict", cxt_dict)

res = (build_chain_without_parsing(model) | StrOutputParser()).invoke(
    {
        "history": cxt_dict,
        "query": """Give the list of all the messages from "user-5" and put them in a "[]" without any instruction text, newlines or additional information.
Notice: Only need content and plan-text.
""",
    }
)
pretty_print("user-5 result", res)

Memory Variables init


{'history': []}

Memory Variables in str list (buffer_as_str) init


''

Memory Variables seeded


...

AIMessage(content='["user-1", "user-2", "user-3", "user-4", "user-5", "user-L", "user-XXXXL"]')

cxt_dict


[
│   "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. \nIf the AI does not know the answer to a question, it truthfully says it does not know.\n\nNotice: The 'uid' is user id, 'role' is user role for human or ai, 'content' is the message content.\n\n",
│   {'role': 'ai', 'content': '{\nuid:""\nrole:"ai"\ncontent:"This is a Gaming Place"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-1"\nrole:"human"\ncontent:"Hello dudes"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-2"\nrole:"human"\ncontent:"hi"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-3"\nrole:"human"\ncontent:"yo yo"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-4"\nrole:"human"\ncontent:"nice to see you"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-5"\nrole:"human"\ncontent:"hoho dude"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-L"\nrole:"human"\ncontent:"o lalala"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-XXXXL"\nrole:"human"\ncontent:"guten tag"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-1"\nrole:"human"\ncontent:"Let\'s get started, ok?"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-2"\nrole:"human"\ncontent:"YES"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-3"\nrole:"human"\ncontent:"YEAH...."\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-4"\nrole:"human"\ncontent:"Cool.."\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-5"\nrole:"human"\ncontent:"yup."\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-L"\nrole:"human"\ncontent:"Great....."\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-XXXXL"\nrole:"human"\ncontent:"alles klar"\n}\n'},
│   {'role': 'human', 'content': '{\nuid:"user-5"\nrole:"human"\ncontent:"I\'m good and the best."\n}\n'},
│   {
│   │   'role': 'human',
│   │   'content': '{\nuid:"user-X"\nrole:"human"\ncontent:"How many users are involved in this conversation, the valid has value in \'uid\'? \nOnly provide the list of user `uid`s in `[... ]` as result without any instruction text or additional information."\n}\n'
│   },
│   {
│   │   'role': 'ai',
│   │   'content': '{\nuid:""\nrole:"ai"\ncontent:"["user-1", "user-2", "user-3", "user-4", "user-5", "user-L", "user-XXXXL"]"\n}\n'
│   }
]

user-5 result


'["hoho dude", "yup.", "I\'m good and the best."]'

#### Failed approach: Convert all history into a list of dictionaries: `cxt_dict`

We can omit compressing the user `role`, `user-id`, and `message` for each dictionary, **but it won't function**.

Different from previous approach,  the  block has `role`, `user-id`, and `message content`.

The issue is akin to the `cxt_history` method. The model input cannot retrieve any user-ids saved in `additional_kwargs`. 

Verify with LangSmith.

In [148]:
memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
memory.buffer.append(HumanMessage(content="alles klar", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Opppsssssss.", id="user-5"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)


def convert_memory_to_dict(memory: ConversationBufferMemory) -> List[Dict[str, str]]:
    """Convert the memory to the dict, role is id, content is the message content."""
    res = [
        """The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. 
If the AI does not know the answer to a question, it truthfully says it does not know.

Notice: The 'uid' is user-id, 'role' is user role for human or ai, 'content' is the message content.

"""
    ]
    history = memory.load_memory_variables({})["history"]
    for hist_item in history:
        role = "human" if isinstance(hist_item, HumanMessage) else "ai"
        res.append(
            {
                "role": role,
                "content": hist_item.content,
                "uid": hist_item.id if role == "human" else "",
            }
        )
    return res


cxt_dict = convert_memory_to_dict(memory)
pretty_print("cxt_dict", cxt_dict)


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = llm
human_query = HumanMessage(
    """How many users are involved in this conversation, the valid has value in 'uid'? 
Only provide the list of user `uid`s in `[... ]` as result.""",
    id="user-X",
)
# human_query = HumanMessage(
#     """How many users are involved in this conversation exclude the AI or System messages?
# Also provide the list of "uid"s. The user ids can be any format unique to each user.

# Notice: 

# Give me a simple result with the only number of users without any instruction text or additional information,
# keep the result as simple as possible,ie. 1,2 or 3....
# Output format: 
# user_count=x, x is number of users

# The user ids will be saved inside "[]".
# Output format: 
# user_ids=[user_1,.......]""",
#     id="user-X",
# )
res = build_chain_without_parsing(model).invoke(
    {
        "history": cxt_dict,
        "query": human_query.content,
    }
)
pretty_print("Result", res)

# Update memory
memory.buffer.append(human_query)
memory.buffer.append(res)
cxt_dict = convert_memory_to_dict(memory)
pretty_print("cxt_dict", cxt_dict)


res = (build_chain_without_parsing(model) | StrOutputParser()).invoke(
    {
        "history": cxt_dict,
        "query": """Give the list of all the messages from the one that "uid" equals "user-5" and put them in a "[]" without any instruction text, newlines or additional information.
Notice: Only need content and plan-text.
""",
    }
)
pretty_print("user-5 result", res)

Memory Variables init


{'history': []}

Memory Variables in str list (buffer_as_str) init


''

Memory Variables seeded


...

AIMessage(content='[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]')

cxt_dict


[
│   "The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. \nIf the AI does not know the answer to a question, it truthfully says it does not know.\n\nNotice: The 'uid' is user-id, 'role' is user role for human or ai, 'content' is the message content.\n\n",
│   {'role': 'ai', 'content': 'This is a Gaming Place', 'uid': ''},
│   {'role': 'human', 'content': 'Hello dudes', 'uid': 'user-1'},
│   {'role': 'human', 'content': 'hi', 'uid': 'user-2'},
│   {'role': 'human', 'content': 'yo yo', 'uid': 'user-3'},
│   {'role': 'human', 'content': 'nice to see you', 'uid': 'user-4'},
│   {'role': 'human', 'content': 'hoho dude', 'uid': 'user-5'},
│   {'role': 'human', 'content': 'o lalala', 'uid': 'user-L'},
│   {'role': 'human', 'content': 'guten tag', 'uid': 'user-XXXXL'},
│   {'role': 'human', 'content': "Let's get started, ok?", 'uid': 'user-1'},
│   {'role': 'human', 'content': 'YES', 'uid': 'user-2'},
│   {'role': 'human', 'content': 'YEAH....', 'uid': 'user-3'},
│   {'role': 'human', 'content': 'Cool..', 'uid': 'user-4'},
│   {'role': 'human', 'content': 'yup.', 'uid': 'user-5'},
│   {'role': 'human', 'content': 'Great.....', 'uid': 'user-L'},
│   {'role': 'human', 'content': 'alles klar', 'uid': 'user-XXXXL'},
│   {'role': 'human', 'content': 'Opppsssssss.', 'uid': 'user-5'},
│   {
│   │   'role': 'human',
│   │   'content': "How many users are involved in this conversation, the valid has value in 'uid'? \nOnly provide the list of user `uid`s in `[... ]` as result.",
│   │   'uid': 'user-X'
│   },
│   {'role': 'ai', 'content': '[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]', 'uid': ''}
]

user-5 result


'["hoho dude", "o lalala"]'

#### (Optional) Use LangChain buildin chain for history based conversation


`ChatMessageHistory` is consturcted by a list of `BaseMessage`. We can convert list of` ("role":"user", "content":"some text")` to list of `BaseMessage`.

In order to be compatible with `convert_openai_messages`, we change the role to openai format, that is ['system', 'assistant', 'user', 'function'], so, we cannot use role as identifier for user.

Any role outside of the above list will cause an error.

`BadRequestError: Error code: 400 - {'error': {'message': "'user-1' is not one of ['system', 'assistant', 'user', 'function'] - 'messages.2.role'", 'type': 'invalid_request_error', 'param': None, 'code': None}}`

```python
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_community.adapters.openai import convert_openai_messages


final_chain = RunnableWithMessageHistory(
    build_chain_without_parsing(model),
    lambda _: ChatMessageHistory(messages=convert_openai_messages(cxt_dict)),
    input_messages_key="query",
    history_messages_key="history",
)
res = final_chain.invoke(
    {"query": human_query.content},
    {"configurable": {"session_id": None}},
)
```

Big advantage: The `ChatMessageHistory` object will grow with each invocation.





In [155]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.memory import ChatMessageHistory
from langchain_community.adapters.openai import convert_openai_messages

memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables init", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str) init", memory.buffer_as_str)

memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables seeded", mem_vars)
pretty_print(
    "Memory Variables in str list (buffer_as_str), seeded", memory.buffer_as_str
)

memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
memory.buffer.append(HumanMessage(content="alles klar", id="user-XL"))
memory.buffer.append(HumanMessage(content="I'm good and the best.", id="user-5"))
memory.buffer.append(HumanMessage(content="so, I am the final one.", id="user-5"))
mem_vars = memory.load_memory_variables({})
pretty_print("Memory Variables", mem_vars)
pretty_print("Memory Variables in str list (buffer_as_str)", memory.buffer_as_str)


def convert_memory_to_dict(memory: ConversationBufferMemory) -> List[Dict[str, str]]:
    """Convert the memory to the dict, role is id, content is the message content."""
    res = list()
    history = memory.load_memory_variables({})["history"]
    content_fmt = """{{
uid:"{uid}"
role:"{role}"
content:"{content}"
}}
"""
    for hist_item in history:
        role = "user" if isinstance(hist_item, HumanMessage) else "assistant"
        res.append(
            {
                "role": role,  # If we want to use identifier for role, we must have openai error: BadRequestError: Error code: 400 - {'error': {'message': "'user-1' is not one of ['system', 'assistant', 'user', 'function'] - 'messages.2.role'", 'type': 'invalid_request_error', 'param': None, 'code': None}}
                "content": content_fmt.format(
                    content=hist_item.content,
                    uid=hist_item.id if role == "user" else None,
                    role=role,
                ),
            }
        )
    return res


cxt_dict = convert_memory_to_dict(memory)
pretty_print("cxt_dict", cxt_dict)


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = llm  # ChatGroq(model="mixtral-8x7b-32768", temperature=0)
human_query = HumanMessage(
    content="""How many users are involved in this conversation, the valid has value in 'uid'? 
Only provide the list of user `uid`s in `[... ]` as result without any instruction text or additional information.""",
    id="user-X",
)

msg_history = ChatMessageHistory(messages=convert_openai_messages(cxt_dict))
pretty_print("msg_history", msg_history)

final_chain = RunnableWithMessageHistory(
    build_chain_without_parsing(model),
    lambda _: msg_history, # auto grow
    input_messages_key="query",
    history_messages_key="history",
)
res = final_chain.invoke(
    {"query": human_query.content},
    {"configurable": {"session_id": None}},
)
pretty_print("Result", res)

pretty_print("msg_history", msg_history)

res = (final_chain | StrOutputParser()).invoke(
    {
        "query": """Give the list of all the messages from the one that "uid" equals "user-5" and put them in a "[]" without any instruction text, newlines or additional information.
Notice: Only need content and plan-text.
""",
    },
    {"configurable": {"session_id": None}},
)
pretty_print("user-5 result", res)

Memory Variables init


{'history': []}

Memory Variables in str list (buffer_as_str) init


''

Memory Variables seeded


...

AIMessage(content='["user-1", "user-2", "user-3", "user-4", "user-5", "user-L", "user-XXXXL", "user-XL"]')

msg_history


ChatMessageHistory(
│   messages=[
│   │   AIMessage(content='{\nuid:"None"\nrole:"assistant"\ncontent:"This is a Gaming Place"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-1"\nrole:"user"\ncontent:"Hello dudes"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-2"\nrole:"user"\ncontent:"hi"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-3"\nrole:"user"\ncontent:"yo yo"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-4"\nrole:"user"\ncontent:"nice to see you"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"user"\ncontent:"hoho dude"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-L"\nrole:"user"\ncontent:"o lalala"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-XXXXL"\nrole:"user"\ncontent:"guten tag"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-1"\nrole:"user"\ncontent:"Let\'s get started, ok?"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-2"\nrole:"user"\ncontent:"YES"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-3"\nrole:"user"\ncontent:"YEAH...."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-4"\nrole:"user"\ncontent:"Cool.."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"user"\ncontent:"yup."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-L"\nrole:"user"\ncontent:"Great....."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-XL"\nrole:"user"\ncontent:"alles klar"\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"user"\ncontent:"I\'m good and the best."\n}\n'),
│   │   HumanMessage(content='{\nuid:"user-5"\nrole:"user"\ncontent:"so, I am the final one."\n}\n'),
│   │   HumanMessage(
│   │   │   content="How many users are involved in this conversation, the valid has value in 'uid'? \nOnly provide the list of user `uid`s in `[... ]` as result without any instruction text or additional information."
│   │   ),
│   │   AIMessage(
│   │   │   content='["user-1", "user-2", "user-3", "user-4", "user-5", "user-L", "user-XXXXL", "user-XL"]'
│   │   )
│   ]
)

user-5 result


'["hoho dude", "yup.", "I\'m good and the best.", "so, I am the final one."]'

#### (Optional) We can pass the query and history in a more elegant way.

Use `RunnableParallel` and `RunnableLambda` to pass data in advance, use `RunnablePassthrough` to directly pass the input query.

```python
(
    {
        "history": RunnableLambda(lambda _: convert_memory_to_dict(memory)),
        "query": RunnablePassthrough(),
    }
    | build_chain_without_parsing(model)
    | StrOutputParser()
).invoke(
    """Give the list of all the messages from the one that "uid" equals "user-5" and put them in a "[]" without any instruction text, newlines or additional information.
Notice: Only need content and plan-text.
"""
)
```

In [195]:
from langchain_core.runnables import (
    RunnableLambda,
    RunnablePassthrough,
)

memory = ConversationBufferMemory(return_messages=True)
mem_vars = memory.load_memory_variables({})


memory.buffer.append(AIMessage(content="This is a Gaming Place"))
mem_vars = memory.load_memory_variables({})


memory.buffer.append(HumanMessage(content="Hello dudes", id="user-1"))
memory.buffer.append(HumanMessage(content="hi", id="user-2"))
memory.buffer.append(HumanMessage(content="yo yo", id="user-3"))
memory.buffer.append(HumanMessage(content="nice to see you", id="user-4"))
memory.buffer.append(HumanMessage(content="hoho dude", id="user-5"))
memory.buffer.append(HumanMessage(content="o lalala", id="user-L"))
memory.buffer.append(HumanMessage(content="guten tag", id="user-XXXXL"))
memory.buffer.append(HumanMessage(content="Let's get started, ok?", id="user-1"))
memory.buffer.append(HumanMessage(content="YES", id="user-2"))
memory.buffer.append(HumanMessage(content="YEAH....", id="user-3"))
memory.buffer.append(HumanMessage(content="Cool..", id="user-4"))
memory.buffer.append(HumanMessage(content="yup.", id="user-5"))
memory.buffer.append(HumanMessage(content="Great.....", id="user-L"))
memory.buffer.append(HumanMessage(content="alles klar", id="user-XL"))
memory.buffer.append(HumanMessage(content="I'm good and the best.", id="user-5"))
memory.buffer.append(HumanMessage(content="so, I am the final one.", id="user-5"))
mem_vars = memory.load_memory_variables({})


def convert_memory_to_dict(memory: ConversationBufferMemory) -> List[Dict[str, str]]:
    """Convert the memory to the dict, role is id, content is the message content."""
    res = list()
    history = memory.load_memory_variables({})["history"]
    content_fmt = """{{
uid:"{uid}"
role:"{role}"
content:"{content}"
}}
"""
    for hist_item in history:
        role = "user" if isinstance(hist_item, HumanMessage) else "assistant"
        res.append(
            {
                "role": role,  # If we want to use identifier for role, we must have openai error: BadRequestError: Error code: 400 - {'error': {'message': "'user-1' is not one of ['system', 'assistant', 'user', 'function'] - 'messages.2.role'", 'type': 'invalid_request_error', 'param': None, 'code': None}}
                "content": content_fmt.format(
                    content=hist_item.content,
                    uid=hist_item.id if role == "user" else None,
                    role=role,
                ),
            }
        )
    return res


def build_chain_without_parsing(
    model: BaseChatModel,
) -> RunnableSerializable[Dict, str]:
    prompt = ChatPromptTemplate.from_messages(
        [
            SystemMessage(
                content=("You are an AI assistant." "You can handle the query of user.")
            ),
            MessagesPlaceholder(variable_name="history"),
            HumanMessagePromptTemplate.from_template("{query}"),
        ]
    )
    return (
        prompt | model
    )  # comment model, you can see the filled template after invoking the chain.


model = ChatGroq(model="llama2-70b-4096", temperature=0, max_tokens=1024 * 4)


final_chain = (
    {
        "history": RunnableLambda(lambda _: convert_memory_to_dict(memory)),
        "query": RunnablePassthrough(),
    }
    | build_chain_without_parsing(model)
    | StrOutputParser()
)
stream = final_chain.astream(
    """Give the list of all the messages from users, no instruction text, newlines, headlines, additional information and decorations in the result. 


Output a list of message content in plan-text only:

Speaker 1(user uid): message content

"""
)
async for chunk in stream:
    print(chunk, end="", flush=True)

user-1: Hello dudes
user-2: hi
user-3: yo yo
user-4: nice to see you
user-5: hoho dude
user-L: o lalala
user-XXXXL: guten tag
user-1: Let's get started, ok?
user-2: YES
user-3: YEAH....
user-4: Cool..
user-5: yup
user-L: Great.....
user-XL: alles klar
user-5: I'm good and the best
user-5: so, I am the final one